In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importing the proteins data
ns3 = []
ns3d = []

f = open("../../../../../../Data/Proteins/DENV4/NS3/DENV4_NS3.txt", "r")
for x in f:
    if "DSS" in x:
        ns3d.append(1)
    elif "DHF" in x:
        ns3d.append(1)
    elif x[0] == ">":
        ns3d.append(0)
    else:
        x = x.replace("\n", "")
        ns3.append(x)

In [3]:
# Converting the arrays into DataFrames
ns3 = pd.DataFrame(ns3)

# Attaching the "Disease" label column to the input
ns3["Disease"] = ns3d

# Renaming the columns
ns3 = ns3.rename(index=str, columns={0: "Sequence", "Disease": "Disease"})

In [4]:
# Concatinating the input data into one DataFrame
print("The combined shape of the given data is:", str(ns3.shape))
print("The length of the combined data is:", str(len(ns3.index)))
print("Does the combined data have any null value? ->", ns3.isnull().values.any())

# Shuffling the data and then taking a peek
ns3 = ns3.sample(frac = 1)
print(ns3.head())

The combined shape of the given data is: (50, 2)
The length of the combined data is: 50
Does the combined data have any null value? -> False
                                             Sequence  Disease
6   SGALWDVPSPAATQKAALSEGVYRIMQRGLFGKTQVGVGIHMEGVF...        0
13  SGALWDVPSPAATQKAALSEGVYRIMQRGLFGKTQVGVGIHMEGVF...        0
48  SGALWDVPSPAAAQKATLTEGVYRIMQRGLFGKTQVGVGIHMEGVF...        1
15  SGALWDVPSPAATQKAALSEGVYRIMQRGLFGKTQVGVGIHMEGVF...        0
23  SGALWDVPSPAATQKAALSEGVYRIMQRGLFGKTQVGVGIHMEGVF...        0


In [5]:
# Function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size = 6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

ns3['words'] = ns3.apply(lambda x: getKmers(x['Sequence']), axis=1)
ns3 = ns3.drop('Sequence', axis=1)

ns3_texts = list(ns3['words'])
for item in range(len(ns3_texts)):
    ns3_texts[item] = ' '.join(ns3_texts[item])

In [6]:
# Creating y and printing the shape of it
y = ns3.iloc[:, 0].values
print("The shape of y is:", y.shape)

The shape of y is: (50,)


In [7]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
x = cv.fit_transform(ns3_texts)

# Print the shape of x
print("The shape of x matrix is:", x.shape)

The shape of x matrix is: (50, 876)


## Nested Cross-Validation

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
# search for an optimal value of C for Logistic Regression
C_list = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 10, 100]
# empty list to store scores
C_scores = []

In [9]:
# 1. we will loop through reasonable values of C
for c in C_list:
    # 2. run SVM with C = c
    svm = SVC(C = c, random_state = 0, kernel = "linear")
    # 3. obtain cross_val_score for SVM with C = c
    scores = cross_val_score(svm, x, y, cv = 10, scoring = "accuracy")
    # 4. append mean of scores for C = c to C_scores list
    C_scores.append(scores.mean())
    
    print("The score for C =", c, "is:", scores.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


The score for C = 0.0001 is: 0.9166666666666667
The score for C = 0.001 is: 0.9166666666666667
The score for C = 0.01 is: 0.8916666666666668


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


The score for C = 0.1 is: 0.8666666666666668
The score for C = 0.5 is: 0.8666666666666668
The score for C = 1.0 is: 0.8666666666666668
The score for C = 10 is: 0.8666666666666668


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


The score for C = 100 is: 0.8666666666666668
